In [22]:
%pip install onnx
%pip install onnxruntime


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
import torch.nn as nn
import torchvision.models as models
from composer.models import ComposerModel
import torchvision
import os
# Instantiate the model


In [24]:
import onnx
model_save_path = os.path.join('./models/143kBPModel.onnx')
onnx_model = onnx.load(model_save_path)
onnx.checker.check_model(onnx_model)

In [53]:
import onnxruntime as ort
import numpy as np
from PIL import Image
import torchvision.transforms as transforms

from torchvision.transforms.functional import InterpolationMode

# Load the image
image_path = "./data/ham10000/Test/nv/ISIC_0024334.jpg"
image = Image.open(image_path).convert('RGB')

# Define the preprocessing steps
transform = transforms.Compose([
            transforms.Resize((300,400), interpolation=InterpolationMode.BILINEAR),
        transforms.CenterCrop((300,400)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # Add any other preprocessing steps if needed, like normalization
])

# Apply the transformations
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Convert tensor to numpy
input_numpy = input_tensor.numpy()
input_numpy_batch = np.repeat(input_numpy, 10, axis=0)
# Load the ONNX model and run inference
ort_session = ort.InferenceSession(model_save_path)

# Get the model's input name
input_name = ort_session.get_inputs()[0].name
# Print the shape of the input tensor
print(f"Shape of input tensor: {input_numpy.shape}")

# Print the expected input shape of the ONNX model
input_shape = ort_session.get_inputs()[0].shape
print(f"Expected input shape of the model: {input_shape}")

# Run inference
outputs = ort_session.run(None, {input_name: input_numpy_batch})

# Print the predicted class
print(f"The predicted class is {np.argmax(outputs[0], axis=1)}")


Shape of input tensor: (1, 3, 300, 400)
Expected input shape of the model: [10, 3, 300, 400]
The predicted class is [5 5 5 5 5 5 5 5 5 5]


In [54]:
class ResNet18(ComposerModel):

    def __init__(self):
        super().__init__()
        self.model = torchvision.models.resnet18()

    def forward(self, batch):  # batch is the output of the dataloader
        # Determine the format of the batch and handle it accordingly
        if isinstance(batch, tuple):
            # If the batch is in the format (inputs, targets)
            inputs, targets = batch
        else:
            # If the batch is just the inputs
            inputs = batch

        return self.model(inputs)

    def loss(self, outputs, batch):
        # pass batches and `forward` outputs to the loss
        _, targets = batch
        return F.cross_entropy(outputs, targets)

In [55]:
model = ResNet18()


In [56]:
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

model.conv1.weight 	 torch.Size([64, 3, 7, 7])
model.bn1.weight 	 torch.Size([64])
model.bn1.bias 	 torch.Size([64])
model.bn1.running_mean 	 torch.Size([64])
model.bn1.running_var 	 torch.Size([64])
model.bn1.num_batches_tracked 	 torch.Size([])
model.layer1.0.conv1.weight 	 torch.Size([64, 64, 3, 3])
model.layer1.0.bn1.weight 	 torch.Size([64])
model.layer1.0.bn1.bias 	 torch.Size([64])
model.layer1.0.bn1.running_mean 	 torch.Size([64])
model.layer1.0.bn1.running_var 	 torch.Size([64])
model.layer1.0.bn1.num_batches_tracked 	 torch.Size([])
model.layer1.0.conv2.weight 	 torch.Size([64, 64, 3, 3])
model.layer1.0.bn2.weight 	 torch.Size([64])
model.layer1.0.bn2.bias 	 torch.Size([64])
model.layer1.0.bn2.running_mean 	 torch.Size([64])
model.layer1.0.bn2.running_var 	 torch.Size([64])
model.layer1.0.bn2.num_batches_tracked 	 torch.Size([])
model.layer1.1.conv1.weight 	 torch.Size([64, 64, 3, 3])
model.layer1.1.bn1.weight 	 torch.Size([64])
model.layer1.1.bn1.bias 	 torch.Size([64])
mode

In [57]:
import torch


model.load_state_dict(torch.load('./models/resnet18BP_20ep.pth', map_location=torch.device('cpu')))


RuntimeError: Error(s) in loading state_dict for ResNet18:
	Missing key(s) in state_dict: "model.layer2.0.conv1.weight", "model.layer2.0.downsample.0.weight", "model.layer3.0.conv1.weight", "model.layer3.0.downsample.0.weight", "model.layer4.0.conv1.weight", "model.layer4.0.downsample.0.weight". 
	Unexpected key(s) in state_dict: "model.maxpool.filt2d", "model.layer2.0.conv1.blur_filter", "model.layer2.0.conv1.conv.weight", "model.layer2.0.downsample.0.blur_filter", "model.layer2.0.downsample.0.conv.weight", "model.layer3.0.conv1.blur_filter", "model.layer3.0.conv1.conv.weight", "model.layer3.0.downsample.0.blur_filter", "model.layer3.0.downsample.0.conv.weight", "model.layer4.0.conv1.blur_filter", "model.layer4.0.conv1.conv.weight", "model.layer4.0.downsample.0.blur_filter", "model.layer4.0.downsample.0.conv.weight". 